In [50]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [51]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [52]:
from src.inference import get_feature_store

In [53]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-02-23 23:19:55,135 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-23 23:19:55,139 INFO: Initializing external client
2025-02-23 23:19:55,140 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-02-23 23:19:55,818 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214632
Fetching data from 2025-01-26 04:19:55.135255+00:00 to 2025-02-24 03:19:55.135255+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.62s) 


In [54]:
feature_group = feature_store.get_feature_group(config.FEATURE_GROUP_NAME, version=config.FEATURE_GROUP_VERSION)
df = feature_group.read()
print(f"Feature Group Record Count: {len(df)}")
df.head()


Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.87s) 
Feature Group Record Count: 2452296


,pickup_hour,pickup_location_id,rides
0,2023-05-25 02:00:00+00:00,88,1
1,2023-06-25 07:00:00+00:00,85,1
2,2023-01-21 05:00:00+00:00,192,0
3,2023-12-12 10:00:00+00:00,221,0
4,2023-10-13 03:00:00+00:00,93,1


In [55]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-01-26 05:00:00+00:00,2,0
1,2025-01-26 06:00:00+00:00,2,0
2,2025-01-26 07:00:00+00:00,2,0
3,2025-01-26 08:00:00+00:00,2,0
4,2025-01-26 09:00:00+00:00,2,0
...,...,...,...
174440,2025-02-23 23:00:00+00:00,263,17
174441,2025-02-24 00:00:00+00:00,263,13
174442,2025-02-24 01:00:00+00:00,263,11
174443,2025-02-24 02:00:00+00:00,263,4


In [56]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [57]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [58]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 174695
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [59]:
ts_data

,pickup_hour,pickup_location_id,rides
0,2025-02-19 19:00:00,255,2
1,2025-01-29 20:00:00,132,161
2,2025-02-23 08:00:00,218,2
3,2025-01-28 00:00:00,67,0
4,2025-02-13 10:00:00,237,369
...,...,...,...
174691,2025-02-23 13:00:00,70,49
174692,2025-01-26 19:00:00,195,0
174693,2025-02-16 17:00:00,203,0
174694,2025-02-12 08:00:00,166,45


In [60]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [61]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,0,3,0,3,5,1,...,0,0,2,0,1,0,0,16,255,2025-01-29 12:00:00
1,161,154,137,68,94,4,234,173,79,95,...,328,13,110,84,327,85,74,63,132,2025-02-05 05:00:00
2,2,1,0,0,0,1,1,0,1,0,...,0,0,0,2,0,0,0,0,218,2025-02-15 00:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,67,2025-02-02 11:00:00
4,369,273,122,247,249,251,17,348,6,17,...,250,5,275,298,1,247,222,393,237,2025-02-17 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,155,2025-02-17 17:00:00
247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,187,2025-01-26 09:00:00
248,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,85,2025-02-14 08:00:00
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,172,2025-02-06 05:00:00


In [62]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,0,0,0,3,0,3,5,1,...,0,0,2,0,1,0,0,16,255,2025-01-29 12:00:00
1,161,154,137,68,94,4,234,173,79,95,...,328,13,110,84,327,85,74,63,132,2025-02-05 05:00:00
2,2,1,0,0,0,1,1,0,1,0,...,0,0,0,2,0,0,0,0,218,2025-02-15 00:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,67,2025-02-02 11:00:00
4,369,273,122,247,249,251,17,348,6,17,...,250,5,275,298,1,247,222,393,237,2025-02-17 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,1,0,0,0,...,0,0,0,1,0,0,0,0,155,2025-02-17 17:00:00
247,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,187,2025-01-26 09:00:00
248,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,1,0,0,1,85,2025-02-14 08:00:00
249,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,172,2025-02-06 05:00:00


In [63]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-02-23 23:20:13,872 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-23 23:20:13,877 INFO: Initializing external client
2025-02-23 23:20:13,877 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-02-23 23:20:14,787 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214632
Fetching data from 2025-01-26 04:20:13.872443+00:00 to 2025-02-24 03:20:13.872443+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.40s) 


In [64]:
current_date

Timestamp('2025-02-24 04:20:13.872443+0000', tz='Etc/UTC')

In [65]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-02-23 05:00:00
1,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,3,2025-02-23 05:00:00
2,0,1,1,0,2,2,6,5,5,3,...,17,26,56,84,117,119,66,18,4,2025-02-23 05:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,6,2025-02-23 05:00:00
4,1,1,2,5,6,3,2,5,2,2,...,4,1,2,2,1,2,4,13,7,2025-02-23 05:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,1,1,1,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,259,2025-02-23 05:00:00
247,0,3,1,1,3,0,2,0,0,3,...,0,0,2,0,0,1,2,4,260,2025-02-23 05:00:00
248,4,0,3,5,11,17,19,26,34,30,...,16,20,25,24,11,6,1,4,261,2025-02-23 05:00:00
249,7,12,33,53,62,103,119,110,144,100,...,32,33,37,12,8,4,8,4,262,2025-02-23 05:00:00


In [66]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-02-23 23:20:24,181 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-02-23 23:20:24,185 INFO: Initializing external client
2025-02-23 23:20:24,185 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-23 23:20:25,274 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214632


In [67]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [68]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,5.0
3,6,0.0
4,7,4.0
...,...,...
246,259,0.0
247,260,1.0
248,261,2.0
249,262,8.0


In [69]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132,  79,  48, 148, 230,  68, 186, 164, 144, 107], dtype=int32)